In [16]:
import gym

In [17]:
import gym
envs = gym.envs.registry
print('Total envs available:', len(envs))

Total envs available: 44


***Random agent*** mountain car problem.

In [18]:
env = gym.make('MountainCar-v0', render_mode="rgb_array")
# Uncomment following line to save video of our Agent interacting in this environment
# This can be used for debugging and studying how our agent is performing
# env = gym.wrappers.Monitor(env, './video/', force = True)
t = 0
while True:
    t += 1
    observation = env.reset()
    env.render()
    print(observation)
    action = env.action_space.sample()
    observation, reward, done, info, _ = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
    break
env.close()

(array([-0.48149312,  0.        ], dtype=float32), {})


### Real solution

In [19]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.optimizers import Adam
from keras import backend as K
import matplotlib.pyplot as plt

EPISODES = 400

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.997
        self.learning_rate = 0.002
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(64, input_dim=2))
        model.add(Activation('relu'))

        model.add(Dense(64))
        model.add(Activation('relu'))

        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss="mean_squared_error",
                      optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future  = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(Q_future)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, name):
        self.model.save(name)

In [20]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

print('state size:' ,state_size)
print('action size: ', action_size)
done = False
batch_size = 128

scores = 0
for e in range(EPISODES):
    print("EPISODE=", e+1)
    state = env.reset()[0]
    
    flag = 0
    for time in range(200):
        # uncomment this to see the actual rendering 
        #env.render()
        action = agent.act(np.reshape(state, (1,2)))
        next_state, reward, done, info, _ = env.step(action)
        
        #print("########################")
        #print("STATE:", type(state), state, state.shape)
        #print("Next state", type(next_state), next_state, next_state.shape)
        #print("########################")

        if next_state[1] > state[1] and next_state[1]>0 and state[1]>0:
            reward += 15
        elif next_state[1] < state[1] and next_state[1]<=0 and state[1]<=0:
            reward += 15
        

        # give more reward if the cart reaches the flag in 200 steps
        if done:
            reward = reward + 10000
        else:
            # put a penalty if the no of time steps is more
            reward = reward - 10  
        agent.remember(np.reshape(state, (1,2)), action, reward, np.reshape(next_state, (1,2)), done)
        state = next_state
        scores += reward
        if done:
            flag = 1
            agent.update_target_model()
            print("episode: {}/{}, score: {}, e: {:.2}"
                    .format(e, EPISODES, scores, agent.epsilon))
            break
        
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    if flag == 0:
        print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))   
    if e % 100 == 0:
        print('saving the model')
        agent.save("mountain_car-dqn.h5")

state size: 2
action size:  3
EPISODE= 1
1/1 [==============================] - 0s 41ms/step
episode: 0/400, score: 199, e: 0.81
saving the model
EPISODE= 2
1/1 [==============================] - 0s 26ms/step


KeyboardInterrupt: 